In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix, hstack
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import warnings
from collections import Counter
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")
from keras.models import Sequential  
from keras.layers import Dropout  
from keras.layers import Dense, Activation  
from keras.utils.np_utils import to_categorical  
  
# coding=utf-8  
from sklearn.preprocessing import StandardScaler # 用于特征的标准化  
from sklearn.preprocessing import Imputer  
from keras.optimizers import RMSprop, Adam, SGD
from keras.utils import np_utils
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

Using TensorFlow backend.


In [4]:
trainFeatures = pd.read_csv('../input2/trainFeatures.csv')
testFeatures = pd.read_csv('../input2/testFeatures.csv')
age_train = pd.read_csv('../input/age_train.csv', header=None)

In [5]:
aggFeatures = pd.read_csv('user_app_date_agg_record.csv')

In [6]:
trainFeatures = trainFeatures.merge(aggFeatures, on='uId', how='left')
testFeatures = testFeatures.merge(aggFeatures, on='uId', how='left')
for column in list(trainFeatures.columns[trainFeatures.isnull().sum() > 0]):
    mean_val = trainFeatures[column].mean()
    trainFeatures[column].fillna(mean_val, inplace=True)
for column in list(testFeatures.columns[testFeatures.isnull().sum() > 0]):
    mean_val = testFeatures[column].mean()
    testFeatures[column].fillna(mean_val, inplace=True)

In [7]:
userAppUsageLabelized = pd.read_csv('user_app_use_most_three_labelize.csv')

In [8]:
trainFeatures = trainFeatures.merge(userAppUsageLabelized, on='uId', how='left')
testFeatures = testFeatures.merge(userAppUsageLabelized, on='uId', how='left')
for column in list(trainFeatures.columns[trainFeatures.isnull().sum() > 0]):
    mode_val = trainFeatures[column].mode()[0]
    trainFeatures[column].fillna(mode_val, inplace=True)
for column in list(testFeatures.columns[testFeatures.isnull().sum() > 0]):
    mode_val = testFeatures[column].mode()[0]
    testFeatures[column].fillna(mode_val, inplace=True)

In [9]:
userAppVec = pd.read_csv('../input2/w2v.csv')

In [10]:
trainFeatures = trainFeatures.merge(userAppVec, on='uId', how='left')
testFeatures = testFeatures.merge(userAppVec, on='uId', how='left')
for column in list(trainFeatures.columns[trainFeatures.isnull().sum() > 0]):
    trainFeatures[column].fillna(0, inplace=True)
for column in list(testFeatures.columns[testFeatures.isnull().sum() > 0]):
    testFeatures[column].fillna(0, inplace=True)

In [ ]:
trainFeatures.to_csv('FinalTrainFeatures.csv', index=False)
testFeatures.to_csv('FinalTestFeatures.csv', index=False)

In [11]:
age_train.columns=['uId', 'age_group']
trainLabels = age_train.age_group

In [ ]:
X_train = trainFeatures.drop(['uId'], axis=1)
y = trainLabels
uId = testFeatures['uId']
X_test = testFeatures.drop('uId', axis=1)

# 特征处理  
sc = StandardScaler()  
sc.fit(X_train)  
X_train = sc.transform(X_train)  
X_test  = sc.transform(X_test)  
y = to_categorical(y) ## 这一步很重要，一定要将多类别的标签进行one-hot编码  
  
'''
model = Sequential()  
model.add(Dense(256, input_shape=(X_train.shape[1],)))  
model.add(Activation('tanh'))  
model.add(Dropout(0.3))  
model.add(Dense(512))  
model.add(Activation('relu'))  
model.add(Dropout(0.3))  
model.add(Dense(512))  
model.add(Activation('tanh'))  
model.add(Dropout(0.3))  
model.add(Dense(256))  
model.add(Activation('linear'))  
model.add(Dense(7)) # 这里需要和输出的维度一致  
model.add(Activation('softmax'))

# For a multi-class classification problem  
model.compile(optimizer='rmsprop',  
              loss='categorical_crossentropy',  
              metrics=['accuracy'])  
'''
  

model = Sequential()
model.add(Dense(1024, input_shape=(X_train.shape[1],)))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Dense(2048))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(7))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.05, decay=5e-4, momentum=0.9, nesterov=True),
              metrics=['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_loss', 
                                            patience = 3,mode='min',
                                            min_delta = 0.001,
                                            verbose = 2, factor=0.3, min_lr = 0)
model.fit(X_train, y, batch_size=256, epochs=50,
          validation_split=0.1,
          callbacks=[learning_rate_reduction],
          shuffle=True)

Train on 1809000 samples, validate on 201000 samples
Epoch 1/50
1809000/1809000 [==============================] - 434s 240us/step - loss: 1.0420 - acc: 0.5520 - val_loss: 0.9986 - val_acc: 0.5746
Epoch 2/50
1809000/1809000 [==============================] - 411s 227us/step - loss: 0.9986 - acc: 0.5712 - val_loss: 0.9814 - val_acc: 0.5801
Epoch 3/50
1809000/1809000 [==============================] - 436s 241us/step - loss: 0.9871 - acc: 0.5762 - val_loss: 0.9754 - val_acc: 0.5833
Epoch 4/50
1809000/1809000 [==============================] - 428s 237us/step - loss: 0.9808 - acc: 0.5789 - val_loss: 0.9725 - val_acc: 0.5848
Epoch 5/50
1809000/1809000 [==============================] - 426s 235us/step - loss: 0.9766 - acc: 0.5807 - val_loss: 0.9709 - val_acc: 0.5855
Epoch 6/50
1809000/1809000 [==============================] - 429s 237us/step - loss: 0.9739 - acc: 0.5824 - val_loss: 0.9665 - val_acc: 0.5876
Epoch 7/50
1809000/1809000 [==============================] - 432s 239us/step - los

In [ ]:
tmp = trainFeatures.drop(['uId'], axis=1)

In [16]:
tmp.head()

,gender,city,prodName,ramCapacity,ramLeftRation,romCapacity,romLeftRation,color,fontSize,ct,...,wordfeature114,wordfeature115,wordfeature116,wordfeature117,wordfeature118,wordfeature119,wordfeature120,wordfeature121,wordfeature122,wordfeature123
0,0,51,78,3.000000,0.43000,32.000000,0.460000,80,1.150000,5,...,4.247619,0.087287,5.330432,8.488958,8.105501,0.660218,1.0,714.0,129.190476,189.354065
1,0,30,138,4.824323,0.34004,82.193725,0.433584,16,1.061655,6,...,4.250000,0.089443,8.007436,8.449906,8.294743,0.114798,1.0,4564.0,402.875000,1143.881164
2,1,228,78,3.000000,0.34000,32.000000,0.060000,80,1.300000,8,...,4.384211,0.474064,7.598722,8.488958,8.248094,0.195258,1.0,8698.0,822.263158,2324.800728
3,0,57,166,2.000000,0.34004,17.000000,0.433584,100,1.061655,7,...,4.300000,0.141421,8.241907,8.274340,8.258123,0.022934,46.0,4312.0,2179.000000,3016.517529
4,1,293,164,2.000000,0.34000,16.000000,0.060000,119,1.000000,8,...,4.230769,0.075107,7.598722,8.438885,8.190766,0.248614,1.0,1044.0,139.000000,311.096716


In [13]:
res = []
for index, case in enumerate(X_test):  
    case = np.array([case])  
    prediction_prob = model.predict(case)
    prediction = np.argmax(prediction_prob)
    res.append(prediction)

KeyboardInterrupt: 

In [ ]:
print(res)

In [ ]:
result = pd.DataFrame({'id': uId, 'label': res})
result.to_csv("keras.csv", index=False)

In [ ]:
res_pro = []
for index, case in enumerate(X_test):  
    case = np.array([case])  
    prediction_prob = model.predict(case)
    res_pro.append(prediction_prob)
    
result = pd.DataFrame({'id': uId, 'pro': res_pro})
result.to_csv("keras_pro.csv", index=False)

In [23]:
res_pro

[array([[  4.64687673e-06,   8.63563560e-04,   1.13110151e-02,
           2.15116560e-01,   7.12451220e-01,   5.43218255e-02,
           5.93111152e-03]], dtype=float32),
 array([[  2.28283170e-06,   1.45173588e-04,   1.98099762e-03,
           2.18793936e-02,   1.50209069e-01,   4.75445807e-01,
           3.50337207e-01]], dtype=float32),
 array([[  1.14633758e-06,   4.77021647e-04,   8.82387757e-02,
           7.57750988e-01,   1.40684634e-01,   1.04233045e-02,
           2.42416002e-03]], dtype=float32),
 array([[  1.00223849e-06,   6.42855593e-04,   4.41268802e-01,
           5.14835835e-01,   3.60010751e-02,   6.53991522e-03,
           7.10494700e-04]], dtype=float32),
 array([[  1.11784073e-04,   2.35919151e-02,   1.75828010e-01,
           2.90094882e-01,   3.02376658e-01,   1.58252776e-01,
           4.97439653e-02]], dtype=float32),
 array([[  8.48145021e-07,   1.37099429e-04,   2.19658054e-02,
           7.66399026e-01,   2.02253416e-01,   5.36847906e-03,
           3.875292